In [1]:
# Imports
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os

warnings.filterwarnings('ignore')


In [161]:
prices_df = pd.read_csv(Path('../resources/prices.csv'))
prices_df['year_month'] = pd.to_datetime(prices_df['year_month'])
prices_df['date'] = pd.to_datetime(prices_df['date'])
prices_df["year"] = prices_df['date'].apply(lambda x: x.year)
prices_df["month"] = prices_df['date'].apply(lambda x: x.month)
prices_df["day"] = prices_df['date'].apply(lambda x: x.day)
prices_df.head()

,year_month,geography,timeframe,date,type,avg_price,total_volume,4046_units,4225_units,4770_units,total_bags,s_bags,l_bags,xl_bags,price_inc,year,month,day
0,2017-01-01,Albany,Weekly,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,7874.69,7866.86,7.83,0.00,1,2017,1,2
1,2017-01-01,Albany,Weekly,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,12674.57,12606.67,67.90,0.00,1,2017,1,8
2,2017-01-01,Albany,Weekly,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,12634.81,12574.72,60.09,0.00,0,2017,1,15
3,2017-01-01,Albany,Weekly,2017-01-22,conventional,1.59,128679.24,4119.94,111173.08,2191.71,11194.51,11060.19,125.50,8.82,1,2017,1,22
4,2017-01-01,Albany,Weekly,2017-01-29,conventional,1.31,95424.59,3844.62,78315.15,484.56,12780.26,12393.84,382.06,4.36,0,2017,1,29


In [162]:
prices_by_GYD = prices_df.groupby(['year_month', 'date','type', 'geography']).sum()['total_volume']
print(prices_by_GYD[0:])
print(prices_by_GYD[54:])

year_month  date        type          geography           
2017-01-01  2017-01-02  conventional  Albany                   129948.23
                                      Atlanta                  547565.88
                                      Baltimore/Washington     631760.81
                                      Boise                    104510.11
                                      Boston                   458830.49
                                                                 ...    
2020-09-01  2020-09-06  organic       Syracuse                   4599.68
                                      Tampa                      4981.81
                                      Total U.S.              1754057.30
                                      West                     315771.42
                                      West Tex/New Mexico       23680.11
Name: total_volume, Length: 20410, dtype: float64
year_month  date        type     geography           
2017-01-01  2017-01-02  organic  A

In [163]:
prices_by_YDT = prices_df.groupby(['year_month', 'date','type']).sum()['total_volume']
prices_by_YDT

year_month  date        type        
2017-01-01  2017-01-02  conventional    1.022769e+08
                        organic         2.171631e+06
            2017-01-08  conventional    1.007855e+08
                        organic         2.649649e+06
            2017-01-15  conventional    1.012323e+08
                                            ...     
2020-08-01  2020-08-23  organic         4.944950e+06
            2020-08-30  conventional    1.340443e+08
                        organic         4.731808e+06
2020-09-01  2020-09-06  conventional    1.350104e+08
                        organic         4.706563e+06
Name: total_volume, Length: 378, dtype: float64

In [164]:
prices_df.nunique()

year_month         45
geography          54
timeframe           1
date              189
type                3
avg_price         255
total_volume    20396
4046_units      19748
4225_units      20011
4770_units      12972
total_bags      20388
s_bags          20360
l_bags          18447
xl_bags          8311
price_inc           2
year                4
month              12
day                31
dtype: int64

In [165]:
prices_df['date'].unique()

array(['2017-01-02T00:00:00.000000000', '2017-01-08T00:00:00.000000000',
       '2017-01-15T00:00:00.000000000', '2017-01-22T00:00:00.000000000',
       '2017-01-29T00:00:00.000000000', '2017-02-05T00:00:00.000000000',
       '2017-02-12T00:00:00.000000000', '2017-02-19T00:00:00.000000000',
       '2017-02-26T00:00:00.000000000', '2017-03-05T00:00:00.000000000',
       '2017-03-12T00:00:00.000000000', '2017-03-19T00:00:00.000000000',
       '2017-03-26T00:00:00.000000000', '2017-04-02T00:00:00.000000000',
       '2017-04-09T00:00:00.000000000', '2017-04-16T00:00:00.000000000',
       '2017-04-23T00:00:00.000000000', '2017-04-30T00:00:00.000000000',
       '2017-05-07T00:00:00.000000000', '2017-05-14T00:00:00.000000000',
       '2017-05-21T00:00:00.000000000', '2017-05-28T00:00:00.000000000',
       '2017-06-04T00:00:00.000000000', '2017-06-11T00:00:00.000000000',
       '2017-06-18T00:00:00.000000000', '2017-06-25T00:00:00.000000000',
       '2017-07-02T00:00:00.000000000', '2017-07-09

In [166]:
print(prices_df['year_month'].min())
print(prices_df['year_month'].max())

2017-01-01 00:00:00
2020-09-01 00:00:00


In [167]:
print(prices_df['date'].min())
print(prices_df['date'].max())

2017-01-02 00:00:00
2020-09-06 00:00:00


In [168]:
climate_df = pd.read_csv(Path('../resources/climate.csv'))
climate_df['year_month'] = pd.to_datetime(climate_df['year_month'])
#climate_df['date'] = pd.to_datetime(climate_df['date'])
climate_df["year"] = climate_df['year_month'].apply(lambda x: x.year)
climate_df["month"] = climate_df['year_month'].apply(lambda x: x.month)
climate_df["day"] = climate_df['year_month'].apply(lambda x: x.day)
climate_df.head()

,year_month,pcp,tavg,pdsi,phdi,zndx,pmdi,cdd,hdd,sp01,...,sp03,sp06,sp09,sp12,sp24,tmin,tmax,year,month,day
0,2010-01-01,6.48,45.2,0.67,-2.02,2.00,-1.14,0,491,0.94,...,0.15,0.25,0.34,0.08,-1.22,36.6,53.7,2010,1,1
1,2010-02-01,3.65,46.8,0.66,-1.75,0.20,-0.76,0,436,0.15,...,0.49,0.23,0.23,-0.11,-1.05,37.5,56.1,2010,2,1
2,2010-03-01,2.19,49.0,0.22,-1.94,-1.12,-1.36,0,420,-0.36,...,0.41,0.14,0.05,-0.05,-0.88,37.2,60.9,2010,3,1
3,2010-04-01,3.47,50.9,1.22,-0.72,3.06,1.03,1,388,1.46,...,0.35,0.31,0.37,0.44,-0.49,39.0,62.8,2010,4,1
4,2010-05-01,1.07,56.9,1.50,1.50,1.22,1.50,14,247,0.49,...,0.55,0.60,0.42,0.42,-0.43,44.0,69.8,2010,5,1


In [169]:
climate_df.nunique()

year_month    130
pcp           101
tavg          110
pdsi          119
phdi          121
zndx          114
pmdi          122
cdd            76
hdd           101
sp01          107
sp02          110
sp03          109
sp06          113
sp09          107
sp12          100
sp24          106
tmin          113
tmax          106
year           11
month          12
day             1
dtype: int64

In [170]:
climate_df['year_month'].unique()

array(['2010-01-01T00:00:00.000000000', '2010-02-01T00:00:00.000000000',
       '2010-03-01T00:00:00.000000000', '2010-04-01T00:00:00.000000000',
       '2010-05-01T00:00:00.000000000', '2010-06-01T00:00:00.000000000',
       '2010-07-01T00:00:00.000000000', '2010-08-01T00:00:00.000000000',
       '2010-09-01T00:00:00.000000000', '2010-10-01T00:00:00.000000000',
       '2010-11-01T00:00:00.000000000', '2010-12-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2011-02-01T00:00:00.000000000',
       '2011-03-01T00:00:00.000000000', '2011-04-01T00:00:00.000000000',
       '2011-05-01T00:00:00.000000000', '2011-06-01T00:00:00.000000000',
       '2011-07-01T00:00:00.000000000', '2011-08-01T00:00:00.000000000',
       '2011-09-01T00:00:00.000000000', '2011-10-01T00:00:00.000000000',
       '2011-11-01T00:00:00.000000000', '2011-12-01T00:00:00.000000000',
       '2012-01-01T00:00:00.000000000', '2012-02-01T00:00:00.000000000',
       '2012-03-01T00:00:00.000000000', '2012-04-01

In [171]:
print(climate_df['year_month'].min())
print(climate_df['year_month'].max())

2010-01-01 00:00:00
2020-10-01 00:00:00


In [172]:
prod_df = pd.read_csv(Path('../resources/prod.csv'))
prod_df['year_month'] = pd.to_datetime(prod_df['year_month'])
prod_df['date'] = pd.to_datetime(prod_df['date'])
prod_df["year"] = prod_df['date'].apply(lambda x: x.year)
prod_df["month"] = prod_df['date'].apply(lambda x: x.month)
prod_df["day"] = prod_df['date'].apply(lambda x: x.day)
prod_df.head()

,year_month,date,status,total_volume,california,chile,mexico,peru,columbia,year,month,day
0,2018-01-01,2018-01-07,actual,45670156,899349,519433,44251374,0,0,2018,1,7
1,2018-01-01,2018-01-14,actual,52758905,830821,449003,51479081,0,0,2018,1,14
2,2018-01-01,2018-01-21,actual,54698266,2913663,445127,51339476,0,0,2018,1,21
3,2018-01-01,2018-01-28,actual,45299201,2698356,95491,42505354,0,0,2018,1,28
4,2018-02-01,2018-02-04,actual,47371481,2635471,0,44736010,0,0,2018,2,4


In [173]:
prod_df.nunique()

year_month       36
date            156
status            2
total_volume    156
california      133
chile            69
mexico          156
peru             68
columbia         17
year              3
month            12
day              31
dtype: int64

In [174]:
prod_df['date'].unique()

array(['2018-01-07T00:00:00.000000000', '2018-01-14T00:00:00.000000000',
       '2018-01-21T00:00:00.000000000', '2018-01-28T00:00:00.000000000',
       '2018-02-04T00:00:00.000000000', '2018-02-11T00:00:00.000000000',
       '2018-02-18T00:00:00.000000000', '2018-02-25T00:00:00.000000000',
       '2018-03-04T00:00:00.000000000', '2018-03-11T00:00:00.000000000',
       '2018-03-18T00:00:00.000000000', '2018-03-25T00:00:00.000000000',
       '2018-04-01T00:00:00.000000000', '2018-04-08T00:00:00.000000000',
       '2018-04-15T00:00:00.000000000', '2018-04-22T00:00:00.000000000',
       '2018-04-29T00:00:00.000000000', '2018-05-06T00:00:00.000000000',
       '2018-05-13T00:00:00.000000000', '2018-05-20T00:00:00.000000000',
       '2018-05-27T00:00:00.000000000', '2018-06-03T00:00:00.000000000',
       '2018-06-10T00:00:00.000000000', '2018-06-17T00:00:00.000000000',
       '2018-06-24T00:00:00.000000000', '2018-07-01T00:00:00.000000000',
       '2018-07-08T00:00:00.000000000', '2018-07-15

In [175]:
print(prod_df['year_month'].min())
print(prod_df['year_month'].max())

2018-01-01 00:00:00
2020-12-01 00:00:00


In [176]:
print(prod_df['date'].min())
print(prod_df['date'].max())

2018-01-07 00:00:00
2020-12-27 00:00:00


In [177]:
prod_df.sort_values(['year_month', 'date'], ascending=True).head(20)

,year_month,date,status,total_volume,california,chile,mexico,peru,columbia,year,month,day
0,2018-01-01,2018-01-07,actual,45670156,899349,519433,44251374,0,0,2018,1,7
1,2018-01-01,2018-01-14,actual,52758905,830821,449003,51479081,0,0,2018,1,14
2,2018-01-01,2018-01-21,actual,54698266,2913663,445127,51339476,0,0,2018,1,21
3,2018-01-01,2018-01-28,actual,45299201,2698356,95491,42505354,0,0,2018,1,28
4,2018-02-01,2018-02-04,actual,47371481,2635471,0,44736010,0,0,2018,2,4
5,2018-02-01,2018-02-11,actual,39307402,3039843,0,36267559,0,0,2018,2,11
6,2018-02-01,2018-02-18,actual,51204346,4264449,0,46939897,0,0,2018,2,18
7,2018-02-01,2018-02-25,actual,53018743,5031685,0,47987058,0,0,2018,2,25
8,2018-03-01,2018-03-04,actual,50628975,4863680,0,45765295,0,0,2018,3,4
9,2018-03-01,2018-03-11,actual,52057338,6816044,0,45241294,0,0,2018,3,11


In [178]:
market_df = pd.read_csv(Path('../resources/market.csv'))
market_df['year_month'] = pd.to_datetime(market_df['year_month'])
market_df['date'] = pd.to_datetime(market_df['date'])
market_df["year"] = market_df['date'].apply(lambda x: x.year)
market_df["month"] = market_df['date'].apply(lambda x: x.month)
market_df["day"] = market_df['date'].apply(lambda x: x.day)
market_df.head()

,year_month,geography,segment,variety,timeframe,period,date,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,year,month,day
0,2017-01-01,Albany,avocados,hass,last 04 wks,13,2017-01-01,394277.71,352292.52,-0.106486,387872.43,438910.27,0.131584,0.983754,1.245869,0.266443,2017,1,1
1,2017-01-01,Atlanta,avocados,hass,last 04 wks,13,2017-01-01,1635155.00,1704001.57,0.042104,1672126.23,1932641.47,0.155799,1.022610,1.134178,0.109101,2017,1,1
2,2017-01-01,Baltimore/Washington,avocados,hass,last 04 wks,13,2017-01-01,2837249.60,2647937.04,-0.066724,3144180.68,3514427.91,0.117756,1.108179,1.327232,0.197670,2017,1,1
3,2017-01-01,Boise,avocados,hass,last 04 wks,13,2017-01-01,245557.30,273591.47,0.114165,270556.76,327836.93,0.211712,1.101807,1.198272,0.087551,2017,1,1
4,2017-01-01,Boston,avocados,hass,last 04 wks,13,2017-01-01,1983011.96,1926040.42,-0.028730,2180519.53,2473499.55,0.134362,1.099600,1.284241,0.167916,2017,1,1


In [179]:
market_df.nunique()

year_month                  45
geography                   54
segment                      1
variety                      1
timeframe                    1
period                      13
date                        49
units_prior_year          2646
units_current_year        2646
unit_variance             2077
dollars_prior_year        2646
dollars_current_year      2646
dollar_variance           2048
avg_price_prior_year      2631
avg_price_current_year    2630
avg_price_variance        2082
year                         4
month                       12
day                         30
dtype: int64

In [180]:
market_df['date'].unique()

array(['2017-01-01T00:00:00.000000000', '2017-01-29T00:00:00.000000000',
       '2017-02-26T00:00:00.000000000', '2017-03-26T00:00:00.000000000',
       '2017-04-23T00:00:00.000000000', '2017-05-21T00:00:00.000000000',
       '2017-06-18T00:00:00.000000000', '2017-07-16T00:00:00.000000000',
       '2017-08-13T00:00:00.000000000', '2017-09-10T00:00:00.000000000',
       '2017-10-08T00:00:00.000000000', '2017-11-05T00:00:00.000000000',
       '2017-12-03T00:00:00.000000000', '2017-12-31T00:00:00.000000000',
       '2018-01-28T00:00:00.000000000', '2018-02-25T00:00:00.000000000',
       '2018-03-25T00:00:00.000000000', '2018-04-22T00:00:00.000000000',
       '2018-05-20T00:00:00.000000000', '2018-06-17T00:00:00.000000000',
       '2018-07-15T00:00:00.000000000', '2018-08-12T00:00:00.000000000',
       '2018-09-09T00:00:00.000000000', '2018-10-07T00:00:00.000000000',
       '2018-11-04T00:00:00.000000000', '2018-12-02T00:00:00.000000000',
       '2018-12-30T00:00:00.000000000', '2019-01-06

In [181]:
print(market_df['year_month'].min())
print(market_df['year_month'].max())

2017-01-01 00:00:00
2020-09-01 00:00:00


In [182]:
print(market_df['date'].min())
print(market_df['date'].max())

2017-01-01 00:00:00
2020-09-06 00:00:00


In [183]:
market_df.sort_values(['year_month', 'geography', 'date'], ascending=True).head(20)

,year_month,geography,segment,variety,timeframe,period,date,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,year,month,day
0,2017-01-01,Albany,avocados,hass,last 04 wks,13,2017-01-01,394277.71,352292.52,-0.106486,387872.43,438910.27,0.131584,0.983754,1.245869,0.266443,2017,1,1
54,2017-01-01,Albany,avocados,hass,last 04 wks,1,2017-01-29,407688.25,414997.59,0.017929,474096.20,526144.34,0.109784,1.162889,1.267825,0.090237,2017,1,29
1,2017-01-01,Atlanta,avocados,hass,last 04 wks,13,2017-01-01,1635155.00,1704001.57,0.042104,1672126.23,1932641.47,0.155799,1.022610,1.134178,0.109101,2017,1,1
55,2017-01-01,Atlanta,avocados,hass,last 04 wks,1,2017-01-29,2007124.88,2254676.72,0.123337,2053575.96,2477949.24,0.206651,1.023143,1.099026,0.074167,2017,1,29
2,2017-01-01,Baltimore/Washington,avocados,hass,last 04 wks,13,2017-01-01,2837249.60,2647937.04,-0.066724,3144180.68,3514427.91,0.117756,1.108179,1.327232,0.197670,2017,1,1
56,2017-01-01,Baltimore/Washington,avocados,hass,last 04 wks,1,2017-01-29,3589693.52,3580774.31,-0.002485,3981610.91,4719902.49,0.185425,1.109179,1.318123,0.188378,2017,1,29
3,2017-01-01,Boise,avocados,hass,last 04 wks,13,2017-01-01,245557.30,273591.47,0.114165,270556.76,327836.93,0.211712,1.101807,1.198272,0.087551,2017,1,1
57,2017-01-01,Boise,avocados,hass,last 04 wks,1,2017-01-29,295379.26,318501.04,0.078278,333403.33,395653.01,0.186710,1.128730,1.242235,0.100560,2017,1,29
4,2017-01-01,Boston,avocados,hass,last 04 wks,13,2017-01-01,1983011.96,1926040.42,-0.028730,2180519.53,2473499.55,0.134362,1.099600,1.284241,0.167916,2017,1,1
58,2017-01-01,Boston,avocados,hass,last 04 wks,1,2017-01-29,2329714.90,2627079.96,0.127640,2663493.79,3285585.89,0.233562,1.143270,1.250661,0.093933,2017,1,29


In [184]:
pri_mar_j_df = prices_df.merge(market_df, how='left', left_on=['year_month', 'date', 'geography'], 
                               right_on=['year_month', 'date', 'geography'])
pri_mar_j_df

,year_month,geography,timeframe_x,date,type,avg_price,total_volume,4046_units,4225_units,4770_units,...,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,year_y,month_y,day_y
0,2017-01-01,Albany,Weekly,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Albany,Weekly,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Albany,Weekly,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Albany,Weekly,2017-01-22,conventional,1.59,128679.24,4119.94,111173.08,2191.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Albany,Weekly,2017-01-29,conventional,1.31,95424.59,3844.62,78315.15,484.56,...,0.017929,474096.20,526144.34,0.109784,1.162889,1.267825,0.090237,2017.0,1.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20405,2020-08-01,West Tex/New Mexico,Weekly,2020-08-09,organic,1.45,30949.07,787.62,2757.96,1658.58,...,0.448115,4181141.94,3782766.99,-0.095279,1.296664,0.810101,-0.375242,2020.0,8.0,9.0
20406,2020-08-01,West Tex/New Mexico,Weekly,2020-08-16,organic,1.53,27717.41,944.69,2917.49,825.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20407,2020-08-01,West Tex/New Mexico,Weekly,2020-08-23,organic,1.47,29346.79,1000.06,1505.03,1765.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20408,2020-08-01,West Tex/New Mexico,Weekly,2020-08-30,organic,1.49,25179.17,936.39,1588.86,1752.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
pri_mar_j_df[pri_mar_j_df['year_month'] == pd.to_datetime("01/01/2017")]

,year_month,geography,timeframe_x,date,type,avg_price,total_volume,4046_units,4225_units,4770_units,...,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,year_y,month_y,day_y
0,2017-01-01,Albany,Weekly,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Albany,Weekly,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Albany,Weekly,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Albany,Weekly,2017-01-22,conventional,1.59,128679.24,4119.94,111173.08,2191.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Albany,Weekly,2017-01-29,conventional,1.31,95424.59,3844.62,78315.15,484.56,...,0.017929,474096.20,526144.34,0.109784,1.162889,1.267825,0.090237,2017.0,1.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20223,2017-01-01,West Tex/New Mexico,Weekly,2017-01-02,organic,1.28,15307.87,867.66,3434.02,37.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20224,2017-01-01,West Tex/New Mexico,Weekly,2017-01-08,organic,1.18,14375.39,1327.98,2617.20,5.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20225,2017-01-01,West Tex/New Mexico,Weekly,2017-01-15,organic,1.19,17014.23,1203.87,2904.22,23.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20226,2017-01-01,West Tex/New Mexico,Weekly,2017-01-22,organic,1.21,16430.64,1413.93,2820.53,20.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
pri_mar_j_df.sort_values(['year_month', 'geography', 'date'], ascending=True)

,year_month,geography,timeframe_x,date,type,avg_price,total_volume,4046_units,4225_units,4770_units,...,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,year_y,month_y,day_y
0,2017-01-01,Albany,Weekly,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10206,2017-01-01,Albany,Weekly,2017-01-02,organic,1.87,1376.70,71.65,192.63,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Albany,Weekly,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10207,2017-01-01,Albany,Weekly,2017-01-08,organic,1.94,2229.52,63.46,478.31,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Albany,Weekly,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20033,2020-09-01,Total U.S.,Weekly,2020-09-06,organic,1.62,1754057.30,105106.37,144137.69,4186.01,...,0.302263,2.066067e+08,2.085794e+08,0.009548,1.270908,0.985241,-0.224774,2020.0,9.0,6.0
7048,2020-09-01,West,Weekly,2020-09-06,conventional,0.95,9026449.59,1975837.16,1152252.54,33199.60,...,0.387555,3.497465e+07,3.620310e+07,0.035124,1.346999,1.004869,-0.253994,2020.0,9.0,6.0
20222,2020-09-01,West,Weekly,2020-09-06,organic,1.58,315771.42,31032.50,28880.42,1192.43,...,0.387555,3.497465e+07,3.620310e+07,0.035124,1.346999,1.004869,-0.253994,2020.0,9.0,6.0
7181,2020-09-01,West Tex/New Mexico,Weekly,2020-09-06,conventional,0.75,1212992.28,397029.46,76486.29,39407.07,...,0.468794,3.717653e+06,3.766063e+06,0.013022,1.137435,0.784485,-0.310304,2020.0,9.0,6.0


In [187]:
pri_join_1 = prices_df.merge(climate_df, how='left', left_on=['year_month'], 
                               right_on=['year_month'])


pri_join_2 = pri_join_1.merge(prod_df, how='left', left_on=['year_month', 'date'], 
                               right_on=['year_month', 'date'])


pri_join_3 = pri_join_2.merge(market_df, how='left', left_on=['year_month', 'geography', 'year', 'month'], 
                               right_on=['year_month', 'geography', 'year', 'month'])

pri_join_3.head(20)


,year_month,geography,timeframe_x,date_x,type,avg_price,total_volume_x,4046_units,4225_units,4770_units,...,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance,day_y
0,2017-01-01,Albany,Weekly,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Albany,Weekly,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Albany,Weekly,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Albany,Weekly,2017-01-22,conventional,1.59,128679.24,4119.94,111173.08,2191.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Albany,Weekly,2017-01-29,conventional,1.31,95424.59,3844.62,78315.15,484.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017-02-01,Albany,Weekly,2017-02-05,conventional,1.49,183549.08,5666.60,165530.03,2119.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017-02-01,Albany,Weekly,2017-02-12,conventional,1.42,97215.94,3808.43,81070.44,175.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017-02-01,Albany,Weekly,2017-02-19,conventional,1.67,95475.07,2702.57,81691.54,125.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-02-01,Albany,Weekly,2017-02-26,conventional,1.40,88371.09,3190.28,73959.76,71.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017-03-01,Albany,Weekly,2017-03-05,conventional,1.18,107354.25,3123.26,90784.49,57.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
pri_join_3.columns

Index(['year_month', 'geography', 'timeframe_x', 'date_x', 'type', 'avg_price',
       'total_volume_x', '4046_units', '4225_units', '4770_units',
       'total_bags', 's_bags', 'l_bags', 'xl_bags', 'price_inc', 'year_x',
       'month_x', 'day_x', 'pcp', 'tavg', 'pdsi', 'phdi', 'zndx', 'pmdi',
       'cdd', 'hdd', 'sp01', 'sp02', 'sp03', 'sp06', 'sp09', 'sp12', 'sp24',
       'tmin', 'tmax', 'year_y', 'month_y', 'day_y', 'status',
       'total_volume_y', 'california', 'chile', 'mexico', 'peru', 'columbia',
       'year', 'month', 'day_x', 'segment', 'variety', 'timeframe_y', 'period',
       'date_y', 'units_prior_year', 'units_current_year', 'unit_variance',
       'dollars_prior_year', 'dollars_current_year', 'dollar_variance',
       'avg_price_prior_year', 'avg_price_current_year', 'avg_price_variance',
       'day_y'],
      dtype='object')

In [189]:
combined_df = pri_join_3.drop(columns=['timeframe_x', 'timeframe_y', 'year_x', 'month_x', 
                                       'day_x', 'year_y', 'month_y', 'day_y', 'year', 'month',
                                      'day_x'], axis=1)





In [190]:
combined_df = combined_df.drop(columns=['date_y'], axis=1)

In [191]:
combined_df.columns

Index(['year_month', 'geography', 'date_x', 'type', 'avg_price',
       'total_volume_x', '4046_units', '4225_units', '4770_units',
       'total_bags', 's_bags', 'l_bags', 'xl_bags', 'price_inc', 'pcp', 'tavg',
       'pdsi', 'phdi', 'zndx', 'pmdi', 'cdd', 'hdd', 'sp01', 'sp02', 'sp03',
       'sp06', 'sp09', 'sp12', 'sp24', 'tmin', 'tmax', 'status',
       'total_volume_y', 'california', 'chile', 'mexico', 'peru', 'columbia',
       'segment', 'variety', 'period', 'units_prior_year',
       'units_current_year', 'unit_variance', 'dollars_prior_year',
       'dollars_current_year', 'dollar_variance', 'avg_price_prior_year',
       'avg_price_current_year', 'avg_price_variance'],
      dtype='object')

In [192]:
combined_df[['date_x', 'total_volume_x', 'total_volume_y']]

,date_x,total_volume_x,total_volume_y
0,2017-01-02,129948.23,NaN
1,2017-01-08,91728.18,NaN
2,2017-01-15,88526.26,NaN
3,2017-01-22,128679.24,NaN
4,2017-01-29,95424.59,NaN
...,...,...,...
21041,2020-08-09,30949.07,52326427.0
21042,2020-08-16,27717.41,45333672.0
21043,2020-08-23,29346.79,54176904.0
21044,2020-08-30,25179.17,48717827.0


In [193]:
combined_df.rename(columns={'date_x': 'date', 'total_volume_x': 'total_volume_price', 'total_volume_y': 'total_volume_prod'}, inplace=True)
combined_df.columns

Index(['year_month', 'geography', 'date', 'type', 'avg_price',
       'total_volume_price', '4046_units', '4225_units', '4770_units',
       'total_bags', 's_bags', 'l_bags', 'xl_bags', 'price_inc', 'pcp', 'tavg',
       'pdsi', 'phdi', 'zndx', 'pmdi', 'cdd', 'hdd', 'sp01', 'sp02', 'sp03',
       'sp06', 'sp09', 'sp12', 'sp24', 'tmin', 'tmax', 'status',
       'total_volume_prod', 'california', 'chile', 'mexico', 'peru',
       'columbia', 'segment', 'variety', 'period', 'units_prior_year',
       'units_current_year', 'unit_variance', 'dollars_prior_year',
       'dollars_current_year', 'dollar_variance', 'avg_price_prior_year',
       'avg_price_current_year', 'avg_price_variance'],
      dtype='object')

In [194]:
combined_df.head()

,year_month,geography,date,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,...,period,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance
0,2017-01-01,Albany,2017-01-02,conventional,1.47,129948.23,4845.77,117027.41,200.36,7874.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Albany,2017-01-08,conventional,1.55,91728.18,3355.47,75641.23,56.91,12674.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Albany,2017-01-15,conventional,1.55,88526.26,3327.65,71956.77,607.03,12634.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Albany,2017-01-22,conventional,1.59,128679.24,4119.94,111173.08,2191.71,11194.51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Albany,2017-01-29,conventional,1.31,95424.59,3844.62,78315.15,484.56,12780.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
combined_df.nunique()

year_month                   45
geography                    54
date                        189
type                          3
avg_price                   255
total_volume_price        20396
4046_units                19748
4225_units                20011
4770_units                12972
total_bags                20388
s_bags                    20360
l_bags                    18447
xl_bags                    8311
price_inc                     2
pcp                          41
tavg                         43
pdsi                         44
phdi                         44
zndx                         43
pmdi                         44
cdd                          31
hdd                          37
sp01                         43
sp02                         40
sp03                         40
sp06                         42
sp09                         41
sp12                         36
sp24                         36
tmin                         44
tmax                         43
status  

In [196]:
combined_df.count()

year_month                21046
geography                 21046
date                      21046
type                      21046
avg_price                 21046
total_volume_price        21046
4046_units                21046
4225_units                21046
4770_units                21046
total_bags                21046
s_bags                    21046
l_bags                    21046
xl_bags                   21046
price_inc                 21046
pcp                       21046
tavg                      21046
pdsi                      21046
phdi                      21046
zndx                      21046
pmdi                      21046
cdd                       21046
hdd                       21046
sp01                      21046
sp02                      21046
sp03                      21046
sp06                      21046
sp09                      21046
sp12                      21046
sp24                      21046
tmin                      21046
tmax                      21046
status  

In [197]:
combined_cleaned_df = combined_df.dropna()
combined_cleaned_df.count()

year_month                14840
geography                 14840
date                      14840
type                      14840
avg_price                 14840
total_volume_price        14840
4046_units                14840
4225_units                14840
4770_units                14840
total_bags                14840
s_bags                    14840
l_bags                    14840
xl_bags                   14840
price_inc                 14840
pcp                       14840
tavg                      14840
pdsi                      14840
phdi                      14840
zndx                      14840
pmdi                      14840
cdd                       14840
hdd                       14840
sp01                      14840
sp02                      14840
sp03                      14840
sp06                      14840
sp09                      14840
sp12                      14840
sp24                      14840
tmin                      14840
tmax                      14840
status  

In [198]:
combined_cleaned_df.head()

,year_month,geography,date,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,...,period,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance
53,2018-01-01,Albany,2018-01-07,conventional,1.13,98540.22,2940.63,76192.61,42.63,19364.35,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
54,2018-01-01,Albany,2018-01-14,conventional,1.42,95246.38,2897.41,76570.67,44.00,15734.30,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
55,2018-01-01,Albany,2018-01-21,conventional,1.69,135196.35,3133.37,116520.88,88.78,15453.32,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
56,2018-01-01,Albany,2018-01-28,conventional,1.57,93625.03,3101.17,74627.23,55.59,15841.04,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
57,2018-02-01,Albany,2018-02-04,conventional,1.03,216738.47,7625.65,195725.06,143.53,13244.23,...,2.0,474327.41,545814.07,0.150712,643283.21,727521.80,0.130951,1.356201,1.332911,-0.017173


In [199]:
for col in combined_cleaned_df.columns:
    print(f"Column {col} has {combined_cleaned_df[col].isnull().sum()} null values")

Column year_month has 0 null values
Column geography has 0 null values
Column date has 0 null values
Column type has 0 null values
Column avg_price has 0 null values
Column total_volume_price has 0 null values
Column 4046_units has 0 null values
Column 4225_units has 0 null values
Column 4770_units has 0 null values
Column total_bags has 0 null values
Column s_bags has 0 null values
Column l_bags has 0 null values
Column xl_bags has 0 null values
Column price_inc has 0 null values
Column pcp has 0 null values
Column tavg has 0 null values
Column pdsi has 0 null values
Column phdi has 0 null values
Column zndx has 0 null values
Column pmdi has 0 null values
Column cdd has 0 null values
Column hdd has 0 null values
Column sp01 has 0 null values
Column sp02 has 0 null values
Column sp03 has 0 null values
Column sp06 has 0 null values
Column sp09 has 0 null values
Column sp12 has 0 null values
Column sp24 has 0 null values
Column tmin has 0 null values
Column tmax has 0 null values
Column 

In [200]:
print(f"Duplicated entries {combined_cleaned_df.duplicated().sum()}")

Duplicated entries 0


In [201]:
combined_cleaned_df['status'].nunique()

1

In [202]:
combined_cleaned_df['segment'].nunique()

1

In [203]:
combined_cleaned_df['variety'].nunique()

1

In [204]:
# type con organic
def change_type_string(str):
        
    if str == "actual":
        return 1
    
    if str == "avocados":
        return 1
    
    if str == "hass":
        return 1
    
    if str == "conventional":
        return 1
    else:
        return 0
    
    
combined_cleaned_df['type'] = combined_cleaned_df['type'].apply(change_type_string)
combined_cleaned_df['status'] = combined_cleaned_df['status'].apply(change_type_string)
combined_cleaned_df['segment'] = combined_cleaned_df['segment'].apply(change_type_string)
combined_cleaned_df['variety'] = combined_cleaned_df['variety'].apply(change_type_string)

combined_cleaned_df.head()

,year_month,geography,date,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,...,period,units_prior_year,units_current_year,unit_variance,dollars_prior_year,dollars_current_year,dollar_variance,avg_price_prior_year,avg_price_current_year,avg_price_variance
53,2018-01-01,Albany,2018-01-07,1,1.13,98540.22,2940.63,76192.61,42.63,19364.35,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
54,2018-01-01,Albany,2018-01-14,1,1.42,95246.38,2897.41,76570.67,44.00,15734.30,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
55,2018-01-01,Albany,2018-01-21,1,1.69,135196.35,3133.37,116520.88,88.78,15453.32,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
56,2018-01-01,Albany,2018-01-28,1,1.57,93625.03,3101.17,74627.23,55.59,15841.04,...,1.0,414997.59,441313.06,0.063411,526144.34,560090.98,0.064520,1.267825,1.269147,0.001042
57,2018-02-01,Albany,2018-02-04,1,1.03,216738.47,7625.65,195725.06,143.53,13244.23,...,2.0,474327.41,545814.07,0.150712,643283.21,727521.80,0.130951,1.356201,1.332911,-0.017173


In [205]:
combined_cleaned_df = pd.get_dummies(data=combined_cleaned_df, columns=['year_month', 'date'])
combined_cleaned_df.head()

,geography,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,s_bags,l_bags,...,date_2020-07-05 00:00:00,date_2020-07-12 00:00:00,date_2020-07-19 00:00:00,date_2020-07-26 00:00:00,date_2020-08-02 00:00:00,date_2020-08-09 00:00:00,date_2020-08-16 00:00:00,date_2020-08-23 00:00:00,date_2020-08-30 00:00:00,date_2020-09-06 00:00:00
53,Albany,1,1.13,98540.22,2940.63,76192.61,42.63,19364.35,8633.09,10707.93,...,0,0,0,0,0,0,0,0,0,0
54,Albany,1,1.42,95246.38,2897.41,76570.67,44.00,15734.30,10012.80,5721.50,...,0,0,0,0,0,0,0,0,0,0
55,Albany,1,1.69,135196.35,3133.37,116520.88,88.78,15453.32,10023.79,5429.53,...,0,0,0,0,0,0,0,0,0,0
56,Albany,1,1.57,93625.03,3101.17,74627.23,55.59,15841.04,11614.79,4159.58,...,0,0,0,0,0,0,0,0,0,0
57,Albany,1,1.03,216738.47,7625.65,195725.06,143.53,13244.23,10571.60,2422.63,...,0,0,0,0,0,0,0,0,0,0


In [206]:
combined_cleaned_df.count()

geography                   14840
type                        14840
avg_price                   14840
total_volume_price          14840
4046_units                  14840
                            ...  
date_2020-08-09 00:00:00    14840
date_2020-08-16 00:00:00    14840
date_2020-08-23 00:00:00    14840
date_2020-08-30 00:00:00    14840
date_2020-09-06 00:00:00    14840
Length: 215, dtype: int64

In [207]:
combined_cleaned_df = combined_cleaned_df.reset_index()

In [208]:
combined_cleaned_df

,index,geography,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,s_bags,...,date_2020-07-05 00:00:00,date_2020-07-12 00:00:00,date_2020-07-19 00:00:00,date_2020-07-26 00:00:00,date_2020-08-02 00:00:00,date_2020-08-09 00:00:00,date_2020-08-16 00:00:00,date_2020-08-23 00:00:00,date_2020-08-30 00:00:00,date_2020-09-06 00:00:00
0,53,Albany,1,1.13,98540.22,2940.63,76192.61,42.63,19364.35,8633.09,...,0,0,0,0,0,0,0,0,0,0
1,54,Albany,1,1.42,95246.38,2897.41,76570.67,44.00,15734.30,10012.80,...,0,0,0,0,0,0,0,0,0,0
2,55,Albany,1,1.69,135196.35,3133.37,116520.88,88.78,15453.32,10023.79,...,0,0,0,0,0,0,0,0,0,0
3,56,Albany,1,1.57,93625.03,3101.17,74627.23,55.59,15841.04,11614.79,...,0,0,0,0,0,0,0,0,0,0
4,57,Albany,1,1.03,216738.47,7625.65,195725.06,143.53,13244.23,10571.60,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14835,21041,West Tex/New Mexico,0,1.45,30949.07,787.62,2757.96,1658.58,25744.91,21405.13,...,0,0,0,0,0,1,0,0,0,0
14836,21042,West Tex/New Mexico,0,1.53,27717.41,944.69,2917.49,825.15,23030.08,19248.24,...,0,0,0,0,0,0,1,0,0,0
14837,21043,West Tex/New Mexico,0,1.47,29346.79,1000.06,1505.03,1765.65,25076.05,21782.09,...,0,0,0,0,0,0,0,1,0,0
14838,21044,West Tex/New Mexico,0,1.49,25179.17,936.39,1588.86,1752.83,20901.09,17579.70,...,0,0,0,0,0,0,0,0,1,0


In [209]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(combined_cleaned_df.geography.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['geography'])
encode_df.head()

,geography_Albany,geography_Atlanta,geography_Baltimore/Washington,geography_Boise,geography_Boston,geography_Buffalo/Rochester,geography_California,geography_Charlotte,geography_Chicago,geography_Cincinnati/Dayton,...,geography_South Carolina,geography_South Central,geography_Southeast,geography_Spokane,geography_St. Louis,geography_Syracuse,geography_Tampa,geography_Total U.S.,geography_West,geography_West Tex/New Mexico
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
encode_df

,geography_Albany,geography_Atlanta,geography_Baltimore/Washington,geography_Boise,geography_Boston,geography_Buffalo/Rochester,geography_California,geography_Charlotte,geography_Chicago,geography_Cincinnati/Dayton,...,geography_South Carolina,geography_South Central,geography_Southeast,geography_Spokane,geography_St. Louis,geography_Syracuse,geography_Tampa,geography_Total U.S.,geography_West,geography_West Tex/New Mexico
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [211]:
# Merge the two DataFrames together and drop the geography column
combined_cleaned_df = combined_cleaned_df.merge(encode_df,left_index=True,right_index=True).drop("geography",1)
combined_cleaned_df.head()

,index,type,avg_price,total_volume_price,4046_units,4225_units,4770_units,total_bags,s_bags,l_bags,...,geography_South Carolina,geography_South Central,geography_Southeast,geography_Spokane,geography_St. Louis,geography_Syracuse,geography_Tampa,geography_Total U.S.,geography_West,geography_West Tex/New Mexico
0,53,1,1.13,98540.22,2940.63,76192.61,42.63,19364.35,8633.09,10707.93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54,1,1.42,95246.38,2897.41,76570.67,44.00,15734.30,10012.80,5721.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,55,1,1.69,135196.35,3133.37,116520.88,88.78,15453.32,10023.79,5429.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56,1,1.57,93625.03,3101.17,74627.23,55.59,15841.04,11614.79,4159.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,57,1,1.03,216738.47,7625.65,195725.06,143.53,13244.23,10571.60,2422.63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
combined_cleaned_df.columns

Index(['index', 'type', 'avg_price', 'total_volume_price', '4046_units',
       '4225_units', '4770_units', 'total_bags', 's_bags', 'l_bags',
       ...
       'geography_South Carolina', 'geography_South Central',
       'geography_Southeast', 'geography_Spokane', 'geography_St. Louis',
       'geography_Syracuse', 'geography_Tampa', 'geography_Total U.S.',
       'geography_West', 'geography_West Tex/New Mexico'],
      dtype='object', length=268)

In [213]:
combined_cleaned_df.count()

index                            14840
type                             14840
avg_price                        14840
total_volume_price               14840
4046_units                       14840
                                 ...  
geography_Syracuse               14840
geography_Tampa                  14840
geography_Total U.S.             14840
geography_West                   14840
geography_West Tex/New Mexico    14840
Length: 268, dtype: int64

In [214]:
combined_cleaned_df.dtypes

index                              int64
type                               int64
avg_price                        float64
total_volume_price               float64
4046_units                       float64
                                  ...   
geography_Syracuse               float64
geography_Tampa                  float64
geography_Total U.S.             float64
geography_West                   float64
geography_West Tex/New Mexico    float64
Length: 268, dtype: object

In [215]:
combined_cleaned_df.to_csv(Path('../resources/combined_cleaned.csv'))